In [18]:
import os
os.makedirs("../docs", exist_ok=True)

# EMBUTIR o plotly.js para não depender de CDN
fig_candle.write_html("../docs/candle.html", include_plotlyjs="inline", full_html=True)
fig_vol.write_html("../docs/volume.html", include_plotlyjs="inline", full_html=True)
fig_sma.write_html("../docs/sma20.html", include_plotlyjs="inline", full_html=True)

# Opcional: garantir que o Pages não rode Jekyll (não é estritamente necessário, mas ajuda)
open("../docs/.nojekyll", "w").close()

print("HTMLs regravados no /docs com Plotly embutido.")


HTMLs regravados no /docs com Plotly embutido.


In [19]:
import pandas as pd, plotly.graph_objects as go, plotly.io as pio, os
pio.renderers.default = "vscode"

# 1) Ler o CSV limpo e garantir tipos
df = pd.read_csv("../data/aapl.csv", parse_dates=[0], index_col=0).sort_index()
df = df[["Open","High","Low","Close","Volume"]].apply(pd.to_numeric, errors="coerce")

# 2) Remover linhas problemáticas (NaN ou OHLC inconsistentes)
df = df.dropna()
df = df[(df["High"]>=df[["Open","Close"]].max(axis=1)) &
        (df["Low"] <=df[["Open","Close"]].min(axis=1))]

# 3) Construir o Candlestick com arrays "puros"
x      = df.index.strftime("%Y-%m-%d").to_numpy()
open_  = df["Open"].astype(float).to_numpy()
high   = df["High"].astype(float).to_numpy()
low    = df["Low"].astype(float).to_numpy()
close  = df["Close"].astype(float).to_numpy()

fig_candle = go.Figure(data=[go.Candlestick(x=x, open=open_, high=high, low=low, close=close)])
fig_candle.update_layout(title="Candlestick", xaxis_rangeslider_visible=False,
                         width=1100, height=600, margin=dict(l=40,r=20,t=60,b=40))

# 4) Salvar página auto-contida (Plotly embutido)
os.makedirs("../docs", exist_ok=True)
from plotly.io import to_html
html = f"""<!doctype html><html lang="pt-br"><head>
<meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1">
<title>Candlestick</title>
<style>body{{font-family:system-ui,Segoe UI,Roboto,Arial;margin:0}}
.wrapper{{max-width:1200px;margin:2rem auto;padding:1rem}}</style>
</head><body><div class="wrapper"><h1>Candlestick</h1>
{to_html(fig_candle, full_html=False, include_plotlyjs="inline")}
<p><a href="./">← Voltar</a></p></div></body></html>"""
with open("../docs/candle.html","w",encoding="utf-8") as f: f.write(html)

# (garante que o GitHub Pages não processe nada)
open("../docs/.nojekyll", "w").close()

print("docs/candle.html recriado.")


docs/candle.html recriado.


In [20]:
import pandas as pd, numpy as np, plotly.graph_objects as go, os, plotly.io as pio
from plotly.io import to_html
pio.renderers.default = "vscode"

# === carregar base limpa ===
df = pd.read_csv("../data/aapl.csv", parse_dates=[0], index_col=0).sort_index()
df = df[["Open","High","Low","Close","Volume"]].apply(pd.to_numeric, errors="coerce").dropna()

# === RSI(14) ===
window = 14
delta = df["Close"].diff()
gain  = delta.clip(lower=0).rolling(window).mean()
loss  = (-delta.clip(upper=0)).rolling(window).mean()
rs    = gain / (loss.replace(0, np.nan))
df["RSI"] = 100 - (100 / (1 + rs))

fig_rsi = go.Figure(go.Scatter(x=df.index, y=df["RSI"], mode="lines", name="RSI(14)"))
fig_rsi.add_hrect(y0=30, y1=70, fillcolor="lightgray", opacity=0.2, line_width=0)
fig_rsi.update_layout(title="RSI(14)", yaxis_title="Índice", width=1100, height=400)

# === MACD (12,26,9) ===
ema12 = df["Close"].ewm(span=12, adjust=False).mean()
ema26 = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = ema12 - ema26
df["Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
df["Hist"] = df["MACD"] - df["Signal"]

fig_macd = go.Figure()
fig_macd.add_scatter(x=df.index, y=df["MACD"],   mode="lines", name="MACD")
fig_macd.add_scatter(x=df.index, y=df["Signal"], mode="lines", name="Signal")
fig_macd.add_bar(x=df.index, y=df["Hist"], name="Hist")
fig_macd.update_layout(title="MACD (12,26,9)", width=1100, height=450)

# === Retorno acumulado ===
ret = df["Close"].pct_change().fillna(0.0)
df["CumRet"] = (1 + ret).cumprod() - 1
fig_ret = go.Figure(go.Scatter(x=df.index, y=df["CumRet"], mode="lines", name="Retorno acumulado"))
fig_ret.update_layout(title="Retorno acumulado (Close)", yaxis_tickformat=".0%", width=1100, height=450)

# === salvar HTMLs no /docs ===
def save_page(fig, path, title):
    fig.update_layout(margin=dict(l=40,r=20,t=60,b=40))
    html = f"""<!doctype html><html lang="pt-br"><head>
<meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1">
<title>{title}</title></head><body style="font-family:system-ui,Segoe UI,Roboto,Arial;margin:0">
<div style="max-width:1200px;margin:2rem auto;padding:1rem">
<h1>{title}</h1>
{to_html(fig, full_html=False, include_plotlyjs="inline")}
<p><a href="./">← Voltar</a></p></div></body></html>"""
    with open(path, "w", encoding="utf-8") as f: f.write(html)

os.makedirs("../docs", exist_ok=True)
save_page(fig_rsi,  "../docs/rsi.html",  "RSI(14)")
save_page(fig_macd, "../docs/macd.html", "MACD (12,26,9)")
save_page(fig_ret,  "../docs/retorno.html", "Retorno acumulado")
open("../docs/.nojekyll","w").close()

print("RSI/MACD/Retorno salvos no /docs")


RSI/MACD/Retorno salvos no /docs


In [28]:
# ========== Observatório Financeiro — Pascoal (Dark/Light + Abas + Coleta Blindada) ==========
import os, numpy as np, pandas as pd
import plotly.graph_objects as go
from pandas_datareader import data as web
from datetime import datetime
from plotly.io import to_html
import plotly.io as pio
import yfinance as yf
import requests
from typing import Optional

# -------------------------------- Config -------------------------------------
TICKERS = ["AAPL", "MSFT", "^BVSP", "PETR4.SA", "VALE3.SA", "ITUB4.SA"]
START, END = datetime(2024, 1, 1), datetime(2024, 8, 31)

WIDTH, HEIGHT = 1100, 600
MARGIN = dict(l=40, r=20, t=60, b=40)
AXGRID = dict(showgrid=True, gridcolor="#334155", zeroline=False)

os.makedirs("../docs", exist_ok=True)

# ------------------------------- Estilo --------------------------------------
def stylize(fig, title=None):
    fig.update_layout(
        # transparente p/ herdar do CSS da página (dark por padrão)
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font=dict(color="#e5e7eb"),
        width=WIDTH, height=HEIGHT, margin=MARGIN,
        title=title or fig.layout.title.text,
    )
    fig.update_xaxes(**AXGRID)
    fig.update_yaxes(**AXGRID)
    return fig

# --------------------------- Ingestão & saneamento ---------------------------
NEEDED = ["open", "high", "low", "close", "volume"]

def _normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip().lower() for c in df.columns]
    aliases = {"vol":"volume","turnover":"volume","adj close":"close","close*":"close"}
    df = df.rename(columns={k:v for k,v in aliases.items() if k in df.columns})
    return df

def _order_and_check(df: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    if df is None or df.empty:
        return None
    df = _normalize_cols(df).sort_index()
    missing = [c for c in NEEDED if c not in df.columns]
    if missing:
        return None
    df = df[NEEDED].apply(pd.to_numeric, errors="coerce").dropna()
    # sanidade OHLC
    df = df[(df["high"] >= df[["open","close"]].max(axis=1)) &
            (df["low"]  <= df[["open","close"]].min(axis=1))]
    return df if not df.empty else None

def _stooq_symbol(tk: str) -> str:
    """Mapeia símbolos problemáticos p/ o equivalente no Stooq (IBOV, etc.)."""
    tku = tk.upper()
    if tku in {"^BVSP", "IBOV", "^IBOV", "BVSP"}:
        return "^BVP"  # índice BOVESPA no Stooq
    return tk

def _strip_b3_suffix(tk: str) -> str:
    """PETR4.SA -> PETR4 (brapi.dev prefere sem sufixo)."""
    return tk.replace(".SA", "").upper()

def _try_stooq(ticker: str) -> Optional[pd.DataFrame]:
    try:
        sym = _stooq_symbol(ticker)
        df = web.DataReader(sym, "stooq", START, END)
        df = _order_and_check(df)
        if df is not None:
            print(f"✓ {ticker}: stooq -> {sym}")
            return df
    except Exception as e:
        print(f"⚠️ {ticker}: stooq falhou: {e}")
    return None

def _try_yfinance(ticker: str) -> Optional[pd.DataFrame]:
    try:
        df = yf.download(ticker, start=START, end=END, progress=False,
                         auto_adjust=False, threads=False)
        return _order_and_check(df)
    except Exception as e:
        print(f"⚠️ {ticker}: yfinance falhou: {e}")
        return None

def _try_yahoo_pdr(ticker: str) -> Optional[pd.DataFrame]:
    try:
        df = web.DataReader(ticker, "yahoo", START, END)
        return _order_and_check(df)
    except Exception as e:
        print(f"⚠️ {ticker}: yahoo falhou: {e}")
        return None

def _try_brapi(ticker: str) -> Optional[pd.DataFrame]:
    """
    Histórico diário via brapi.dev (sem API key p/ ranges modestos).
    Válido para B3 (ex.: PETR4.SA, VALE3.SA, ITUB4.SA).
    """
    if not ticker.upper().endswith(".SA"):
        return None
    sym = _strip_b3_suffix(ticker)
    url = f"https://brapi.dev/api/quote/{sym}?range=1y&interval=1d&fundamental=false&dividends=false"
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        js = r.json()
        hist = js.get("results", [{}])[0].get("historicalDataPrice", [])
        if not hist:
            return None
        df = pd.DataFrame(hist)
        if "date" in df.columns:
            df["date"] = pd.to_datetime(df["date"], unit="s", utc=True).dt.tz_localize(None)
            df = df.set_index("date")
        # padroniza & recorta período
        rename = {"open":"open","high":"high","low":"low","close":"close","volume":"volume"}
        df = df.rename(columns=rename)
        df = df[list(rename.values())]
        df = df[(df.index >= pd.Timestamp(START)) & (df.index <= pd.Timestamp(END))]
        return _order_and_check(df)
    except Exception as e:
        print(f"⚠️ {ticker}: brapi.dev falhou: {e}")
        return None

def _try_csv_local(ticker: str) -> Optional[pd.DataFrame]:
    for p in (f"../data/{ticker}.csv",
              f"../data/{ticker.replace('.SA','')}.csv",
              f"../data/{ticker.upper()}.csv",
              f"../data/{ticker.replace('.SA','').upper()}.csv"):
        if os.path.exists(p):
            try:
                df = pd.read_csv(p, parse_dates=[0], index_col=0)
                df = _order_and_check(df)
                if df is not None:
                    print(f"✓ {ticker}: csv local -> {os.path.basename(p)}")
                    return df
            except Exception as e:
                print(f"⚠️ {ticker}: csv local falhou ({p}): {e}")
    return None

def fetch_any(ticker: str) -> Optional[pd.DataFrame]:
    for fn in (_try_stooq, _try_yfinance, _try_yahoo_pdr, _try_brapi, _try_csv_local):
        df = fn(ticker)
        if df is not None:
            return df
    print(f"⛔ {ticker}: sem dados válidos (pulando).")
    return None

# -------------------------------- Figuras ------------------------------------
def fig_candle(df, title):
    fig = go.Figure(go.Candlestick(
        x=df.index, open=df["open"], high=df["high"], low=df["low"], close=df["close"]
    ))
    fig.update_layout(xaxis_rangeslider_visible=False)
    return stylize(fig, title)

def fig_volume(df):
    fig = go.Figure(go.Bar(x=df.index, y=df["volume"]))
    fig.update_layout(yaxis_title="Volume")
    return stylize(fig, "Volume")

def fig_sma(df, window=20):
    d = df.copy()
    d["sma"] = d["close"].rolling(window, min_periods=1).mean()
    fig = go.Figure()
    fig.add_scatter(x=d.index, y=d["close"], name="Close", mode="lines")
    fig.add_scatter(x=d.index, y=d["sma"],   name=f"SMA {window}", mode="lines")
    fig.update_layout(yaxis_title="Preço")
    return stylize(fig, f"Média Móvel {window}d")

def fig_rsi(df, window=14):
    delta = df["close"].diff()
    gain  = delta.clip(lower=0).rolling(window).mean()
    loss  = (-delta.clip(upper=0)).rolling(window).mean().replace(0, np.nan)
    rsi   = 100 - (100/(1 + (gain/loss)))
    f = go.Figure(go.Scatter(x=df.index, y=rsi, name=f"RSI({window})", mode="lines"))
    f.add_hrect(y0=30, y1=70, fillcolor="#475569", opacity=0.25, line_width=0)
    return stylize(f, f"RSI({window})")

def fig_macd(df):
    ema12  = df["close"].ewm(span=12, adjust=False).mean()
    ema26  = df["close"].ewm(span=26, adjust=False).mean()
    macd   = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    hist   = macd - signal
    f = go.Figure()
    f.add_scatter(x=df.index, y=macd,   name="MACD",   mode="lines")
    f.add_scatter(x=df.index, y=signal, name="Signal", mode="lines")
    f.add_bar(x=df.index, y=hist, name="Hist")
    return stylize(f, "MACD (12,26,9)")

def fig_bollinger(df, window=20, k=2):
    ma = df["close"].rolling(window).mean()
    std = df["close"].rolling(window).std()
    upper, lower = ma + k*std, ma - k*std
    f = go.Figure()
    f.add_scatter(x=df.index, y=df["close"], name="Close", mode="lines")
    f.add_scatter(x=df.index, y=upper, name=f"BBand+{k}σ", mode="lines")
    f.add_scatter(x=df.index, y=ma,    name=f"SMA {window}", mode="lines")
    f.add_scatter(x=df.index, y=lower, name=f"BBand-{k}σ", mode="lines")
    return stylize(f, f"Bollinger Bands ({window},{k}σ)")

def fig_atr(df, window=14):
    prev_close = df["close"].shift(1)
    tr = pd.concat([
        (df["high"] - df["low"]),
        (df["high"] - prev_close).abs(),
        (df["low"]  - prev_close).abs()
    ], axis=1).max(axis=1)
    atr = tr.rolling(window).mean()
    f = go.Figure(go.Scatter(x=df.index, y=atr, mode="lines", name="ATR"))
    return stylize(f, f"ATR({window})")

def fig_heatmap_monthly_returns(df):
    # 'M' -> 'ME' (MonthEnd) para evitar deprecation
    mret = df["close"].resample("ME").last().pct_change().dropna()
    tab = (mret.to_frame("ret")
           .assign(Ano=lambda x: x.index.year,
                   MesNum=lambda x: x.index.month))
    pivot = (tab.pivot(index="Ano", columns="MesNum", values="ret")
                .sort_index(axis=1).fillna(0))
    nomes_pt = ["Jan","Fev","Mar","Abr","Mai","Jun","Jul","Ago","Set","Out","Nov","Dez"]
    pivot.columns = [nomes_pt[i-1] for i in pivot.columns]

    import plotly.express as px
    f = px.imshow(pivot*100, text_auto=".1f", aspect="auto",
                  labels=dict(color="%"), color_continuous_scale="RdYlGn")
    return stylize(f, "Retornos mensais (%)")

# ------------------ UI base (CSS/JS) Dark + abas + Plotly CDN ----------------
PLOTLY_CDN = "https://cdn.plot.ly/plotly-2.35.2.min.js"

BASE_CSS = """
:root{--bg:#0b1020;--card:#0f172a;--text:#e5e7eb;--muted:#9ca3af;--border:#1f2937;--accent:#8b5cf6}
[data-theme="light"]{--bg:#ffffff;--card:#ffffff;--text:#111827;--muted:#6b7280;--border:#e5e7eb;--accent:#7c3aed}
*{box-sizing:border-box}
body{margin:0;font-family:system-ui,Segoe UI,Roboto,Arial;background:var(--bg);color:var(--text)}
.wrapper{max-width:1300px;margin:2rem auto;padding:1rem}
.header{display:flex;align-items:center;gap:1rem;justify-content:space-between}
h1{margin:.2rem 0}
.muted{color:var(--muted)}
.card{background:var(--card);border:1px solid var(--border);border-radius:14px;padding:1rem;box-shadow:0 1px 4px rgba(0,0,0,.25)}
a{color:var(--accent);text-decoration:none}a:hover{text-decoration:underline}
.tabs{display:flex;gap:.5rem;flex-wrap:wrap;margin:1rem 0}
.tab{border:1px solid var(--border);background:transparent;color:var(--text);padding:.5rem .8rem;border-radius:10px;cursor:pointer}
.tab.active{background:var(--accent);border-color:var(--accent);color:#fff}
.panel{display:none;margin-top:.5rem}
.panel.active{display:block}
.grid{display:grid;gap:1rem}
@media(min-width:1100px){.grid.two{grid-template-columns:1fr 1fr}}
.toggle{display:inline-flex;gap:.5rem;align-items:center;border:1px solid var(--border);padding:.35rem .6rem;border-radius:999px;cursor:pointer}
"""

THEME_JS = f"""
<script>
(function(){{
  const key='theme';
  const root=document.documentElement;
  function applyTheme(t){{
    root.setAttribute('data-theme', t);
    const dark = (t==='dark');
    const rel = dark ? {{
      paper_bgcolor:'rgba(0,0,0,0)', plot_bgcolor:'rgba(0,0,0,0)',
      font:{{color:'#e5e7eb'}}, xaxis:{{gridcolor:'#334155'}}, yaxis:{{gridcolor:'#334155'}}
    }} : {{
      paper_bgcolor:'rgba(255,255,255,0)', plot_bgcolor:'rgba(255,255,255,0)',
      font:{{color:'#111827'}}, xaxis:{{gridcolor:'#e5e7eb'}}, yaxis:{{gridcolor:'#e5e7eb'}}
    }};
    if (window.Plotly) {{
      document.querySelectorAll('.plotly-graph-div').forEach(gd=>{{ try{{ Plotly.relayout(gd, rel); }}catch(e){{}} }});
    }}
  }}
  const saved = localStorage.getItem(key) || 'dark';
  applyTheme(saved);
  window.__toggleTheme = function(){{
    const t = (root.getAttribute('data-theme')==='dark')?'light':'dark';
    localStorage.setItem(key,t); applyTheme(t);
  }};
}})();
</script>
"""

TABS_JS = """
<script>
function activateTab(groupId, tabId){
  document.querySelectorAll(`[data-group="${groupId}"] .tab`).forEach(b=>b.classList.remove('active'));
  document.querySelectorAll(`[data-group="${groupId}"] .panel`).forEach(p=>p.classList.remove('active'));
  document.getElementById(`${groupId}-tab-${tabId}`).classList.add('active');
  document.getElementById(`${groupId}-panel-${tabId}`).classList.add('active');
}
</script>
"""

def fig_to_div(fig) -> str:
    return to_html(fig, full_html=False, include_plotlyjs=False)

# ------------------- Página por ticker (abas internas) -----------------------
def write_ticker_page(ticker: str, sections: list[tuple[str, str, str]]):
    tabs = "\n".join(
        f'<button id="tabs-{ticker}-tab-{slug}" class="tab" onclick="activateTab(\'tabs-{ticker}\', \'{slug}\')">{title}</button>'
        for slug, title, _ in sections
    )
    panels = "\n".join(
        f'<div id="tabs-{ticker}-panel-{slug}" class="panel{" active" if i==0 else ""}">{html}</div>'
        for i, (slug, _title, html) in enumerate(sections)
    )

    html = f"""<!doctype html><html lang="pt-br">
<head>
  <meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1">
  <title>{ticker} — Painel</title>
  <script src="{PLOTLY_CDN}"></script>
  <style>{BASE_CSS}</style>
  {THEME_JS}
  {TABS_JS}
</head>
<body>
  <div class="wrapper">
    <div class="header">
      <div>
        <h1>{ticker} — Painel</h1>
        <div class="muted">Plotly • Stooq/Yahoo • GitHub Pages</div>
      </div>
      <button class="toggle" onclick="__toggleTheme()">Alternar tema</button>
    </div>

    <div class="card">
      <div class="tabs" data-group="tabs-{ticker}">
        {tabs}
      </div>
      {panels}
      <p style="margin-top:1rem"><a href="../">← Home</a></p>
    </div>
  </div>

  <script>activateTab('tabs-{ticker}', '{sections[0][0]}');</script>
</body></html>"""
    with open(f"../docs/{ticker}.html", "w", encoding="utf-8") as f:
        f.write(html)

# --------------------------- Home (cartões) ----------------------------------
def write_home(cards):
    items = "\n".join(
        f'<a class="card" href="{tk}.html" style="text-decoration:none"><h2>{tk}</h2><div class="muted">Abrir painel completo</div></a>'
        for tk in cards
    )
    html = f"""<!doctype html><html lang="pt-br">
<head>
  <meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1">
  <style>{BASE_CSS}</style>
  {THEME_JS}
  <title>Observatório Financeiro</title>
</head>
<body>
  <div class="wrapper">
    <div class="header">
      <div>
        <h1>Observatório Financeiro</h1>
        <div class="muted">Plotly • Stooq/Yahoo • GitHub Pages</div>
      </div>
      <button class="toggle" onclick="__toggleTheme()">Alternar tema</button>
    </div>
    <div class="grid two">{items}</div>
    <p class="muted" style="margin-top:2rem">Feito por Pasquara — ciência com método, arte com medida.</p>
  </div>
</body></html>"""
    with open("../docs/index.html", "w", encoding="utf-8") as f:
        f.write(html)

# ---------------------------- Build por ticker --------------------------------
built = []
for tk in TICKERS:
    df = fetch_any(tk)
    if df is None:
        continue

    sections = []
    sections.append(("candle", "Candlestick",       fig_to_div(fig_candle(df, f"{tk} — Candlestick"))))
    sections.append(("volume", "Volume",            fig_to_div(fig_volume(df))))
    sections.append(("sma20",  "Média Móvel 20d",   fig_to_div(fig_sma(df, 20))))
    sections.append(("rsi",    "RSI(14)",           fig_to_div(fig_rsi(df))))
    sections.append(("macd",   "MACD (12,26,9)",    fig_to_div(fig_macd(df))))
    sections.append(("boll",   "Bollinger (20,2σ)", fig_to_div(fig_bollinger(df, 20, 2))))
    sections.append(("atr",    "ATR(14)",           fig_to_div(fig_atr(df, 14))))

    ret = df["close"].pct_change().fillna(0)
    cum = (1 + ret).cumprod() - 1
    f_ret = go.Figure(go.Scatter(x=df.index, y=cum, mode="lines", name="Ret Acum"))
    f_ret.update_yaxes(tickformat=".0%")
    sections.append(("ret", "Retorno acumulado",    fig_to_div(stylize(f_ret, "Retorno acumulado"))))

    sections.append(("heat",  "Heatmap mensal (%)", fig_to_div(fig_heatmap_monthly_returns(df))))

    write_ticker_page(tk, sections)
    built.append(tk)

# ------------------------------ Home + Pages ----------------------------------
write_home(built)
open("../docs/.nojekyll", "w").close()
print("✅ Gerado: /docs/index.html e /docs/<TICKER>.html (tema Dark/Light e abas).")


✓ AAPL: stooq -> AAPL
✓ MSFT: stooq -> MSFT
✓ ^BVSP: stooq -> ^BVP
Failed to get ticker 'PETR4.SA' reason: Expecting value: line 1 column 1 (char 0)
- PETR4.SA: No data found for this date range, symbol may be delisted

1 Failed download:
- PETR4.SA: No data found for this date range, symbol may be delisted
⚠️ PETR4.SA: yahoo falhou: Unable to read URL: https://finance.yahoo.com/quote/PETR4.SA/history?period1=1704092400&period2=1725173999&interval=1d&frequency=1d&filter=history
Response Text:
b'<html><meta charset=\'utf-8\'>\n<script>\nif(window != window.top){\ndocument.write(\'<p>Content is currently unavailable.</p><img src="//geo.yahoo.com/p?s=1197757039&t=\'\n    + new Date().getTime() + \'&_R=\'\n    + encodeURIComponent(document.referrer)\n    + \'&err=404\'\n    + \'" width="0px" height="0px"/>\');\n}else{\nwindow.location.replace(\'https://\' + window.location.host + \'/?err=404\');\n}\n</script>\n<noscript>\n<META http-equiv="refresh" content="0;URL=https://finance.yahoo.com/